In [1]:
import os
import torch
import torchvision
import numpy as np
import pickle
import mne
import scipy
import matplotlib.pyplot as plt
from scipy.io import loadmat
from sklearn.preprocessing import StandardScaler, RobustScaler, MaxAbsScaler

In [2]:
a = np.load('/well/woolrich/projects/replay/study2/epoched/subj1/cond0/trial0.npy')

In [3]:
a.shape

(401, 240)

In [ ]:
# check if noshuffle data with pca is same as pca306 data
# load data
noshuffle_dir = '/well/woolrich/projects/cichy118_cont/preproc_data_onepass/lowpass25hz/subj0/train_data_noshuffle/'
pca_data_dir = '/well/woolrich/projects/cichy118_cont/preproc_data_onepass/lowpass25hz/subj0/train_data_pca306/'

noshuffle = []
for ch in range(306):
    path = noshuffle_dir + 'cch' + str(ch) + '.mat'
    noshuffle.append(loadmat(path)['x_train_t'][:, 0, :])

pca_data = []
for ch in range(306):
    path = pca_data_dir + 'cch' + str(ch) + '.mat'
    pca_data.append(loadmat(path)['x_train_t'][:, 0, :])

pca_data = np.array(pca_data)

path = '/well/woolrich/projects/cichy118_cont/preproc_data_onepass/lowpass25hz/subj0/train_data_pca306/pca'
with open(path, 'rb') as f:
    pca = pickle.load(f)
path = '/well/woolrich/projects/cichy118_cont/preproc_data_onepass/lowpass25hz/subj0/train_data_pca306/norm'
with open(path, 'rb') as f:
    norm = pickle.load(f)

noshuffle = np.array(noshuffle).transpose(1, 2, 0).reshape(-1, 306)

# apply pca and norm
x_train = pca.transform(noshuffle)
x_train = norm.transform(x_train)

x_train = x_train.reshape(-1, 101, 306)

plt.plot(np.average(x_train[:, :, 73], axis=0))
plt.plot(np.average(pca_data[73, :, :], axis=0))


In [ ]:
x_train.shape

In [ ]:
# check ordering of cichy92
path = '/well/woolrich/projects/cichy92/preprocessed/125hz_lowpass'

paths = os.listdir(path)
for p in paths:
    print(p)

In [ ]:
path = '/well/woolrich/projects/cichy118_cont/preproc_data_osl/cont_quantized_clamp8/ch0.mat'
data = loadmat(path)

In [ ]:
data['x_train_t'].shape

In [ ]:
onehot = np.zeros(256)
onehot[120] = 1

kernel = torchvision.transforms.GaussianBlur(3, sigma=(0.1))

In [ ]:
filtered = scipy.ndimage.gaussian_filter1d(onehot, sigma=2, axis=0, order=0, output=None, mode='reflect', truncate=4.0)
plt.plot(filtered)

In [ ]:
filtered[120]

In [ ]:
plt.plot(onehot)

In [ ]:
path = '/gpfs2/well/woolrich/projects/cichy118_cont/preproc_data_onepass/lowpass25hz/subj1/cond0/trial0.npy'
trial = np.load(path)

In [ ]:
trial.shape

In [ ]:
cov = np.load(f"/well/woolrich/projects/cichy118_cont/preproc_data_onepass/cont/subj0_cov.npy")

In [ ]:
np.mean(cov)

In [ ]:
# create a random matrix of size 3x2
x = torch.rand(3, 2)

In [ ]:
x

In [ ]:
x.reshape(-1)

In [ ]:
weights = torch.tensor([0.1, 0.2, 0.6, 0.1], dtype=torch.float) # create a tensor of weights
torch.multinomial(weights, 2)

In [ ]:
chn_ids.shape

In [ ]:
cond = torch.rand(2, 4, 3)
cond_ind = torch.randint(low=0, high=4, size=(2, 1, 3))
cond_ind = (cond_ind > 0).float()

In [ ]:
cond_ind

In [ ]:
cond

In [ ]:
cond*cond_ind

In [ ]:
targets = np.random.rand(3, 4, 2)
chind = np.random.randint(low=0, high=4, size=(3))

In [ ]:
targets

In [ ]:
chind

In [ ]:
targets[[0,1,2], chind, :].shape

In [ ]:
bins = np.linspace(-1, 1, 255 + 1)
digitized = np.digitize([1, 0.1, 0.2, -0.5], bins) - 1

In [ ]:
signal = np.array([1, 0.1, 0.2, -0.5, -1])
digitized = ((signal + 1) / 2 * 255+0.5).astype(np.int32)
recon = 2 * ((digitized).astype(np.float32) / 255) - 1

In [ ]:
recon

In [ ]:
username = input("Enter username:")

## Checking quantized data

In [ ]:
def mulaw_inv(x, maxabs, mu=255):
    x = (x - 0.5) / mu * 2 - 1
    x = np.sign(x)*((1+mu)**np.abs(x)-1)/mu

    x = x * maxabs.scale_[0]

    return x


In [ ]:
maxabs_path = '/well/woolrich/projects/cichy118_cont/preproc_data_osl/subj1/cont_quantized_quant8/maxabs_scaler'
# load scaler from file
maxabs = pickle.load(open(maxabs_path, 'rb'))

In [ ]:
maxabs.scale_

In [ ]:
path = '/well/woolrich/projects/cichy118_cont/preproc_data_osl/subj1/subj1.npy'
dataset = np.load(path)
dataset = dataset[0, ::4]
dataset = RobustScaler().fit_transform(dataset.reshape(-1, 1)).reshape(-1)

In [ ]:
path = '/well/woolrich/projects/cichy118_cont/preproc_data_osl/subj1/cont_quantized_quant8/'
data = loadmat(path + 'ch0.mat')['x_val_t'][0]
target = loadmat(path + 'ch306.mat')['x_val_t'][0]
cond = loadmat(path + 'ch612.mat')['x_val_t'][0]
subid = loadmat(path + 'ch613.mat')['x_val_t'][0]

In [ ]:
# extract decoding examples
shifted = cond[1:] - cond[:-1]
inds = np.where(shifted > 0)[0] - 24

In [ ]:
inds.shape

In [ ]:
decoded = mulaw_inv(data, maxabs, mu=7)

In [ ]:
np.average(np.abs((decoded - dataset[:len(decoded)])/dataset[:len(decoded)]))

In [ ]:
%matplotlib widget
plt.plot(dataset[:1000], linewidth=0.5)
plt.plot(decoded[:1000], linewidth=0.5)
#plt.plot((decoded[:100000] - dataset[:100000]))


In [ ]:
%matplotlib widget
plt.plot(cond[:10000])

In [ ]:
%matplotlib widget
plt.hist(data, bins=100)
plt.show()

## Checking outputs from generative decoder

In [ ]:
outputs = torch.Tensor(np.load(os.path.join('cichy_data', 'subj1_quantized', 'outputs.npy')))
targets = torch.Tensor(np.load(os.path.join('cichy_data', 'subj1_quantized', 'targets.npy')))
targets = targets.long()

targets = targets.reshape(8, -1)
outputs = outputs.reshape(8, -1, outputs.shape[1])

In [ ]:
loss = torch.nn.CrossEntropyLoss(reduction='none')

In [ ]:
out_inds = torch.argmax(outputs,dim=-1)

In [ ]:
out_inds[5]

In [ ]:
targets[5]

In [ ]:
plt.plot(out_inds[2], 'r', targets[2], 'b')

In [ ]:
for i in range(8):
    loss_val = torch.mean(loss(outputs[i], targets[i].long()))
    print(loss_val.item())

In [ ]:
torch.mean(loss(outputs.reshape(-1, outputs.shape[2]), targets.reshape(-1)))

# Checking output

In [ ]:
path = '../results/cichy_epoched/subj1/cont_quantized_1_35hz/wavenetfull_rf1s_dim2/trainlosses.npy'
losses = np.load(path)

In [ ]:
losses.shape

In [ ]:
ch_loss = np.mean(losses, axis=(0, 2))

In [ ]:
%matplotlib widget
plt.plot(ch_loss)

In [ ]:
# Load data
path = '/well/woolrich/projects/cichy118_cont/preproc_data_osl/subj1/oslpy/MEG2_subj02_sess01_tsss_mc-0_preproc_raw.fif'
raw = mne.io.read_raw_fif(path, preload=True)
chn_type = 'meg'
raw = raw.pick(chn_type)

In [ ]:
evoked_ch1 = mne.EvokedArray(ch_loss.reshape(-1, 1), raw.info, tmin=0)

In [ ]:
%matplotlib widget
evoked_ch1.plot_topomap(scalings=1, vmin=2)

In [ ]:
path = '/well/woolrich/projects/cichy118_cont/preproc_data_osl/subj1/subj1.npy'
dataset = np.load(path)
dataset = dataset[:, ::4]
dataset = RobustScaler().fit_transform(dataset.T)

In [ ]:
dataset.shape

In [ ]:
%matplotlib widget
for i in range(9,10):
    plt.plot(dataset[:, i])

plt.hlines([3, -3], 0, 1e6)

In [ ]:
diffs = []
for i in range(1, 11):
    clipped = np.clip(dataset, -i, i)
    diffs.append(np.average(np.abs(clipped-dataset)))

In [ ]:
%matplotlib widget
plt.bar(np.arange(1, 11), height=diffs, log=True)

In [ ]:
eye = np.eye(10)

inds = np.random.randint(0, 10, (2,3,4))

selected = eye[inds]
print(inds[:, 0, 0])
selected[:, 0, 0]


In [ ]:
path = '/well/woolrich/projects/cichy118_cont/preproc_data_osl/subj2/subj2.npy'
data = np.load(path)
data.shape

In [ ]:
path = '/well/woolrich/projects/cichy118_cont/preproc_data_osl/subj2/event_times.npy'
ev = np.load(path)

In [ ]:
ev[-1]

In [ ]:
[154000, 48000, 85000, 70000, 57000, 67000, 49000, 89000, 187000, 56000, 37000, 106000, 81000, 83000, 240000]

In [ ]:
subj = 15
path = f'/well/woolrich/projects/cichy118_cont/raw_data/subj{subj:02d}/MEG2_subj{subj:02d}_sess01_tsss_mc-0.fif'
raw = mne.io.read_raw_fif(path)